In [1]:
# 1、加载模型和预训练权重
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = r"F:\pythonProject\google_bert\bert_base_uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at F:\pythonProject\google_bert\bert_base_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

看到这么一大串的warning出现，不要怕，这个warning正是我们希望看到的。

为啥会出现这个warning呢，因为我们加载的预训练权重是bert-based-uncased，而使用的骨架是AutoModelForSequenceClassification，前者是没有在下游任务上微调过的，所以用带有下游任务Head的骨架去加载，会随机初始化这个Head。这些在warning中也说的很明白。

In [4]:
# 2、加载数据集
from datasets import load_from_disk

dataset_path = r"F:\pythonProject\datasets\glue\mrpc"
raw_datasets = load_from_disk(dataset_path)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [8]:
# 3、查看整个数据集结构
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

load_dataset出来的是一个DatasetDict对象，它包含了train，validation，test三个属性。可以通过key来直接查询，得到对应的train、valid和test数据集。

这里的train，valid，test都是Dataset类型，有 features和num_rows两个属性。还可以直接通过下标来查询对应的本。

In [5]:
# 3.1查看单个样本数据结构
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [7]:
# 3.2查看单个样本的特征
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

Dataset的features可以理解为一张表的columns，Dataset甚至可以看做一个pandas的dataframe，二者的使用很类似。

In [12]:
# 4、数据集预处理
def tokenize_function(sample):
    # 这里可以添加多种操作，不光是tokenize
    # 这个函数处理的对象，就是Dataset这种数据类型，通过features中的字段来选择要处理的数据
    return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

官方推荐的做法是通过Dataset.map方法，来调用一个分词方法，实现批量化的分词。注意到，在这个tokenize_function中，没有使用padding，因为如果使用了padding之后，就会全局统一做一个maxlen进行padding，这样无论在tokenize还是模型的训练上都不够高效。实际上，我们是故意先不进行padding的，因为我们想在划分batch的时候再进行padding，这样可以避免出现很多有一堆padding的序列，从而可以显著节省我们的训练时间。这样就需要用到**DataCollatorWithPadding，来进行动态padding**

In [13]:
# 4.1 动态padding，collator数据
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='F:\pythonProject\google_bert\bert_base_uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_

注意，我们需要使用tokenizer来初始化这个DataCollatorWithPadding，因为需要tokenizer来告知具体的padding token是啥，
以及padding的方式是在左边还是右边（不同的预训练模型，使用的padding token以及方式可能不同）。

In [14]:
# 5、实例化trainer对象
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir='test_trainer') # 指定输出文件夹，没有会自动创建

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,  # 在定义了tokenizer之后，其实这里的data_collator就不用再写了，会自动根据tokenizer创建
    tokenizer=tokenizer,
)

E:\Anaconda\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# 5.1启动训练
trainer.train()

In [ ]:
# 5.2 
trainer.predict()

trainer.predict()函数处理的结果是一个named_tuple（一种可以直接通过key来取值的tuple），类似一个字典，包含三个属性：predictions, label_ids, metrics
* predictions实际上就是logits* 
label_ids不是预测出来的id，而是数据集中自带的ground truth的标签，因此如果输入的数据集中没给标签，这里也不会输
* metrics，也是只有输入的数据集中提供了label_ids才会输出metrics，包括loss之类的指标参数。

其中metrics中还可以包含我们自定义的字段，我们需要在定义Trainer的时候给定compute_metrics参数。